# 🎯 Simple Draft Tracker

**One job: Track drafted players and update your data files.**

- Select player → Mark as drafted → Updates all your sheets automatically
- No complicated features, just simple player tracking

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load your data
cheat_sheet = pd.read_csv('draft_cheat_sheet.csv')
cheat_sheet['DRAFTED'] = False  # Add drafted column

print(f"📊 Loaded {len(cheat_sheet)} players")
print("Ready to track drafts!")

In [ ]:
# Simple Draft Interface
class SimpleDraftTracker:
    def __init__(self):
        self.setup_widgets()
        
    def setup_widgets(self):
        # Player search and selection
        self.search_box = widgets.Text(
            placeholder='Search player...',
            description='Find:',
            style={'description_width': 'initial'}
        )
        
        self.player_list = widgets.Select(
            options=[],
            rows=10,
            description='Available:',
            style={'description_width': 'initial'}
        )
        
        # Draft button
        self.draft_btn = widgets.Button(
            description='✅ DRAFT PLAYER',
            button_style='success',
            layout={'width': '200px', 'height': '50px'}
        )
        
        # Undo button
        self.undo_btn = widgets.Button(
            description='↩️ UNDO LAST',
            button_style='warning'
        )
        
        # Save button
        self.save_btn = widgets.Button(
            description='💾 SAVE DATA',
            button_style='info'
        )
        
        # Output areas
        self.status_out = widgets.Output()
        self.drafted_out = widgets.Output()
        
        # Event handlers
        self.search_box.observe(self.update_list, names='value')
        self.draft_btn.on_click(self.draft_player)
        self.undo_btn.on_click(self.undo_draft)
        self.save_btn.on_click(self.save_data)
        
        # Track drafted players
        self.draft_history = []
        
        # Initial setup
        self.update_list()
        self.show_drafted()
        
    def update_list(self, change=None):
        """Update available players list"""
        available = cheat_sheet[cheat_sheet['DRAFTED'] == False].copy()
        
        # Filter by search
        if self.search_box.value:
            search = self.search_box.value.lower()
            available = available[available['Player'].str.lower().str.contains(search, na=False)]
        
        # Create options
        options = []
        for _, player in available.head(20).iterrows():
            label = f"{player['Draft_Rank']:2d}. {player['Player']} ({player['Position']}, {player['Team']})"
            options.append(label)
        
        self.player_list.options = options
        
    def draft_player(self, btn):
        """Mark selected player as drafted"""
        if not self.player_list.value:
            with self.status_out:
                clear_output()
                print("❌ No player selected")
            return
            
        # Extract player name from selection
        selected = self.player_list.value
        player_name = selected.split('. ')[1].split(' (')[0]
        
        # Mark as drafted
        player_idx = cheat_sheet[cheat_sheet['Player'] == player_name].index
        if len(player_idx) > 0:
            cheat_sheet.loc[player_idx[0], 'DRAFTED'] = True
            self.draft_history.append(player_name)
            
            with self.status_out:
                clear_output()
                print(f"✅ DRAFTED: {player_name}")
                
            self.update_list()
            self.show_drafted()
        
    def undo_draft(self, btn):
        """Undo last draft"""
        if not self.draft_history:
            with self.status_out:
                clear_output()
                print("❌ No drafts to undo")
            return
            
        # Undo last player
        last_player = self.draft_history.pop()
        player_idx = cheat_sheet[cheat_sheet['Player'] == last_player].index
        if len(player_idx) > 0:
            cheat_sheet.loc[player_idx[0], 'DRAFTED'] = False
            
            with self.status_out:
                clear_output()
                print(f"↩️ UNDRAFTED: {last_player}")
                
            self.update_list()
            self.show_drafted()
            
    def save_data(self, btn):
        """Save updated data files"""
        with self.status_out:
            clear_output()
            
            # Save main cheat sheet
            cheat_sheet.to_csv('draft_cheat_sheet_updated.csv', index=False)
            
            # Create available players file
            available = cheat_sheet[cheat_sheet['DRAFTED'] == False]
            available.to_csv('available_players.csv', index=False)
            
            # Create drafted players file
            drafted = cheat_sheet[cheat_sheet['DRAFTED'] == True]
            drafted.to_csv('drafted_players.csv', index=False)
            
            print("💾 SAVED:")
            print("  - draft_cheat_sheet_updated.csv")
            print("  - available_players.csv")
            print("  - drafted_players.csv")
            print(f"\n📊 {len(available)} available, {len(drafted)} drafted")
    
    def show_drafted(self):
        """Show recently drafted players"""
        with self.drafted_out:
            clear_output()
            
            drafted = cheat_sheet[cheat_sheet['DRAFTED'] == True]
            total_drafted = len(drafted)
            total_available = len(cheat_sheet) - total_drafted
            
            print(f"📈 DRAFT STATUS: {total_drafted} drafted, {total_available} available\n")
            
            if total_drafted > 0:
                print("🏈 RECENTLY DRAFTED:")
                recent = self.draft_history[-10:] if len(self.draft_history) > 10 else self.draft_history
                for i, player in enumerate(reversed(recent), 1):
                    print(f"  {i:2d}. {player}")
            else:
                print("No players drafted yet")
    
    def display(self):
        """Display the interface"""
        controls = widgets.VBox([
            widgets.HTML("<h3>🎯 Quick Draft Tracker</h3>"),
            self.search_box,
            self.player_list,
            widgets.HBox([self.draft_btn, self.undo_btn, self.save_btn]),
            self.status_out
        ])
        
        return widgets.HBox([controls, self.drafted_out])

# Create and show tracker
tracker = SimpleDraftTracker()
display(tracker.display())

## 🎯 How It Works:

1. **Search** for player by name
2. **Select** from the list 
3. **Click "✅ DRAFT PLAYER"** - done!
4. **"💾 SAVE DATA"** creates updated files for other notebooks

### Files Created:
- `draft_cheat_sheet_updated.csv` - Your original file with DRAFTED column
- `available_players.csv` - Only undrafted players  
- `drafted_players.csv` - Only drafted players

**Simple. Fast. Gets the job done.** 🏆